In [5]:
# Ainda não finalizei/rodei o projeto, apenas estruturei a partir de um código que tinha aqui. 
# To revisando VAE ainda.

In [4]:
# Importar as bibliotecas necessárias
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Carregar o dataset Wine
data = load_wine()
x = data['data']  # Características
y = data['target']  # Rótulos
feature_names = data['feature_names']

# Dividir os dados em treino e validação
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Normalizar os dados (entre 0 e 1)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [ ]:
# Dimensão do espaço latente
latent_dim = 2

# Encoder
encoder_inputs = tf.keras.Input(shape=(x_train.shape[1],))
x = layers.Dense(16, activation="relu")(encoder_inputs)
x = layers.Dense(8, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)

# Reparametrização (Sampling)
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,), name="z")([z_mean, z_log_var])
encoder = tf.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
# Decoder
latent_inputs = tf.keras.Input(shape=(latent_dim,))
x = layers.Dense(8, activation="relu")(latent_inputs)
x = layers.Dense(16, activation="relu")(x)
decoder_outputs = layers.Dense(x_train.shape[1], activation="sigmoid")(x)
decoder = tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
# Definir o modelo VAE
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            # Passar pelos modelos
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            # Reconstruction Loss
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.square(data - reconstruction), axis=1))
            # KL Divergence
            kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            kl_loss = tf.reduce_mean(kl_loss)
            total_loss = reconstruction_loss + kl_loss
        # Backpropagation
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {"loss": total_loss, "reconstruction_loss": reconstruction_loss, "kl_loss": kl_loss}

vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())

In [ ]:
# Treinamento
history = vae.fit(x_train, x_train, epochs=50, batch_size=16, validation_data=(x_val, x_val))

In [ ]:
# Projeção do espaço latente
z_mean, _, _ = encoder.predict(x_val)

# Reduzir para 2D (se necessário)
pca = PCA(n_components=2)
z_pca = pca.fit_transform(z_mean)

# Gráfico do espaço latente
plt.figure(figsize=(8, 6))
scatter = plt.scatter(z_pca[:, 0], z_pca[:, 1], c=y_val, cmap="viridis")
plt.colorbar(scatter, label="Classes")
plt.title("Projeção do Espaço Latente (PCA)")
plt.xlabel("Latente 1")
plt.ylabel("Latente 2")
plt.show()

# Variância explicada pelo PCA
print(f"Variância explicada pelo PCA: {pca.explained_variance_ratio_}")